# Segmenting the right part of the CT image

In this short notebook, I want to show you a rather easy way to segement the right part of CT image including just the skull in the middle of the resulting image. This helps your model focus better in the region of interest and also reduce the image size without losing much information. **Please upvote if you found it helpful**

I'll be using some code and strategies for correcting bad dicoms files and loading the df from notebooks by [akensert](http://https://www.kaggle.com/akensert/rsna-inceptionv3-keras-tf1-14-0) and [Jeremy Howard](http://https://www.kaggle.com/jhoward/cleaning-the-data-for-rapid-prototyping-fastai). Thank you so much for sharing your great works!

In [ ]:
import os
import cv2
import glob
import pydicom
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train.csv")
df[['ID', 'type']] = df['ID'].str.rsplit("_", n=1, expand=True)
df.drop_duplicates(['ID', 'type'], inplace=True)
df = df.pivot('ID', 'type', 'Label')
df.reset_index(inplace=True)
df.head()

**Let's take a look at some of the original images**

In [ ]:
train_path = "../input/rsna-intracranial-hemorrhage-detection/rsna-intracranial-hemorrhage-detection/stage_2_train/"

def visualize(imgs, axes=None, rows=2, cols=5, **kwargs):
    if axes is None: _, axes = plt.subplots(rows, cols, **kwargs)
    for img, ax in zip(imgs, axes.flatten()):
        ax.imshow(img, cmap='bone')
        ax.axis("off")

sample_df = df.sample(10, random_state=8)
sample_df_fnames = [train_path + f"{id_}.dcm" for id_ in sample_df['ID'].values]

dcms = [pydicom.dcmread(fname) for fname in sample_df_fnames]
arrs = [dcm.pixel_array for dcm in dcms]
visualize(arrs, figsize=(15, 5))

In the next section, I will use the code snippet I found [here](http://https://stackoverflow.com/questions/59865712/segment-photos-of-bio-samples-to-extract-circular-area-of-interest-with-python-i) for a similar problem and I change it a liitle bit to segment the main circular (does not have to be exactly a circle) object in the image. I also return the coordinates and width and height to able to crop the other windows of the CT with the same numbers in order to get fully overlapping images. I will give this function the brain windowed image and use the mentioned numbers to crop other windowed images of the same CT. 

In [ ]:
def segment_circle(windowed):
    original = windowed.copy().astype("uint8")
    mask = np.zeros(original.shape, dtype=np.uint8)
    gray = original
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    close = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=5)
    cnts = cv2.findContours(close, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.04 * peri, True)
        area = cv2.contourArea(c)
        if len(approx) > 4 and area > 10000 and area < 500000:
            ((x, y), r) = cv2.minEnclosingCircle(c)
            cv2.circle(mask, (int(x), int(y)), int(r), (255, 255, 255), -1)
            cv2.circle(original, (int(x), int(y)), int(r), (36, 255, 12), 0)
    x,y,w,h = cv2.boundingRect(mask)
    mask_ROI = mask[y:y+h, x:x+w]
    image_ROI = original[y:y+h, x:x+w]
    result = cv2.bitwise_and(image_ROI, image_ROI, mask=mask_ROI)
    return result, (x, y, w, h)

I'll use these functions to correct the bad dicoms and window the CT (see the notebooks I put the refernce to at the beginning my notebook)

In [ ]:
def correct_dcm(dcm):
    x = dcm.pixel_array + 1000
    px_mode = 4096
    x[x>=px_mode] = x[x>=px_mode] - px_mode
    dcm.PixelData = x.tobytes()
    dcm.RescaleIntercept = -1000

def window_image(img, window_center, window_width):
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    img = np.clip(img, img_min, img_max)
    return img

**Here is the main function which reads the dicoms, windows them and then segments accordingly**

In [ ]:
def preprocess_and_segment(path, norm='min_max', make_correct=True):    
    dcm = pydicom.dcmread(path)
    if make_correct and (dcm.BitsStored == 12) and (dcm.PixelRepresentation == 0) and (int(dcm.RescaleIntercept) > -100):
        correct_dcm(dcm)

    sample = dcm.pixel_array.astype("float32") * dcm.RescaleSlope + dcm.RescaleIntercept
    
    brain_window = window_image(sample, 40, 80)
    dural_window = window_image(sample, 80, 200)
    bone_window = window_image(sample, 600, 2800)
    
    brain_window, (x,y,w,h) = segment_circle(brain_window)
    if brain_window is None:
        return np.zeros((512, 512, 3))
    dural_window = dural_window[y:y+h, x:x+w]
    bone_window = bone_window[y:y+h, x:x+w]
    
    # Just some normalization. You can ignore this
    if norm == "min_max":
        brain_window = (brain_window - (0.)) / 80.
        dural_window = (dural_window - (-20.)) / 200.
        bone_window = (bone_window - (-1200.)) / 2800.
        img_3ch = np.dstack([brain_window, dural_window, bone_window]).astype("float32")
    elif norm == "hu":
        img_3ch = np.dstack([brain_window, dural_window, bone_window]).astype("float32") / 4096.
    elif norm == 'none':
        img_3ch = np.dstack([brain_window, dural_window, bone_window]).astype("float32")
    
    return img_3ch

**The original images (again, for comparison)**

In [ ]:
sample_df = df.sample(10, random_state=8)
sample_df_fnames = [train_path + f"{id_}.dcm" for id_ in sample_df['ID'].values]

dcms = [pydicom.dcmread(fname) for fname in sample_df_fnames]
arrs = [dcm.pixel_array for dcm in dcms]
visualize(arrs, figsize=(15, 5))

**After segmenting (cropping) the right part of the image**

In [ ]:
proccessed = [preprocess_and_segment(fname, norm="min_max") for fname in sample_df_fnames]
visualize([img[..., 0] for img in proccessed], figsize=(15, 5))

As you can see, the segmentation function successfully cropped the right part of the image and the skull is in the middle of the resulting images. When the brain tissue is really low or in some other rare situations, the segmentation function could fail (which is not frequent and happens mostly when the image label is 0 with no hemorrhage) and returns **None** which in this case I'm retuning a black image (np array full of zeros). You can see an example where the function failed to segment in the first image of second row.